# Votaciones Diputados Argentina Data Discovery

En este Jupyter Notebook, vamos a descubrir metadata sobre el dataset de las votaciones de legisladores argentinos. En este [link](https://github.com/nahuelhds/votaciones-diputados-argentina) se pueden encontrar más detalles sobre esto.

## Data Discovery
Para explorar los datos se utilizará [Pandas](https://pandas.pydata.org/), una librería de Python muy conocida. 

Antes de empezar, algunos imports...

In [1]:
import os
import urllib.request
import pandas as pd
import plotly as py

Y se declaran algunas variables útiles:

In [2]:
import_directory = os.path.join("/tmp", "datasets", "votaciones")

dataset_base_url = "https://raw.githubusercontent.com/nahuelhds/votaciones-diputados-argentina"
dataset_path = "/master/csv/"
full_path = dataset_base_url + dataset_path

# You can use as many datasets as you want here. They'll all be cached in your file system.
datasets = [
    "votaciones_votos-2015.csv",
    "votaciones_votos-2016.csv",
    "votaciones_votos-2017.csv",
    "votaciones_votos-2018.csv"
]

Generamos metadata temporal sobre los datasets. Esta metadata es útil a la hora de descargar y cachear los datasets en el file system.

In [3]:

# { dataset: { url: http://..., file_name: /tmp/datasets/...} }
datasets_metadata = {}

# Populate dataset metadata.
for dataset in datasets:
    dataset_file_name = os.path.join(import_directory, dataset)
    dataset_url = full_path + dataset
    datasets_metadata.update({dataset: {
        'url': dataset_url,
        'file_name': dataset_file_name
    }})

In [4]:
def download_dataset( dataset_url, file_name):
    print("Downloading " + dataset_url + " into " + file_name)
    urllib.request.urlretrieve(dataset_url, file_name)


In [5]:
def download_datasets():
    print("Downloading Datasets from " + full_path)
    for dataset in datasets:
        dataset_metadata = datasets_metadata[dataset]
        download_dataset(dataset_metadata.get('url'), dataset_metadata.get('file_name'))
    print("Datasets downloaded")


In [6]:
def load_datasets():
    dfs = []
    for dataset in datasets:
        dataset_filename = datasets_metadata[dataset].get('file_name')
        if not dfs:
            # first dataset with the headers
            df = pd.read_csv(dataset_filename)
        else:
            # the rest without headers
            df = pd.read_csv(dataset_filename, index_col=None, header=0)
        dfs.append(df)
    final_df = pd.concat(dfs, axis=0, ignore_index=True)
    return final_df

In [7]:
if not os.path.isdir(import_directory):
    print("Creating tmp directory")
    os.makedirs(import_directory)
    download_datasets()

In [8]:
votos = load_datasets()
print("##### Votos de ejemplo:#######")
print(votos.head())
print("\n#######Resumen del dataset######")
print(votos.info())

##### Votos de ejemplo:#######
       id  voting_id  legislador_id                         legislador  \
0  458450       1785           1156  ABDALA DE MATARAZZO, Norma Amanda   
1  458451       1785           1337                 ABRAHAM, Alejandro   
2  458452       1785            930                  AGUAD, Oscar Raúl   
3  458453       1785           1247               AGUILAR, Lino Walter   
4  458454       1785           1338             ALEGRE, Gilberto Oscar   

   bloque_id                        bloque  provincia_id            provincia  \
0        119    Frente Cívico por Santiago            22  Santiago del Estero   
1          1  Frente para la Victoria - PJ            13              Mendoza   
2          6          Unión Cívica Radical             3              Córdoba   
3        197            Compromiso Federal            19             San Luis   
4         85              Frente Renovador             1         Buenos Aires   

         voto  
0  AFIRMATIVO  
1  AF

Usando `plotly`, se pueden visualizar los votos del bloque o de un legislador:

In [9]:
# set up de plotly
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
init_notebook_mode(connected=True)

In [11]:
# Creamos tmp folder
if not os.path.exists('images'):
    os.mkdir('images')

In [68]:
# De todos los campos solo queremos bloque y voto
subset = votos.filter(["bloque", "voto"])

positivos = subset['voto'] == 'AFIRMATIVO'
negativos = subset['voto'] == 'NEGATIVO'
ausentes = subset['voto'] == 'AUSENTE'

In [69]:
# creamos 3 datasets: afirmativo, negativo y ausente
positivos_por_bloque = subset[positivos].groupby('bloque')
negativos_por_bloque = subset[negativos].groupby('bloque')
ausentes_por_bloque = subset[ausentes].groupby('bloque')

In [70]:
# Realizamos el cuento de votos por bloque y graficamos
positivos_count = positivos_por_bloque.agg('count')

labels = positivos_count.index.values
values = positivos_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Positivos')

py.offline.iplot([trace], filename='positivos')

In [63]:
negativos_count = negativos_por_bloque.agg('count')

labels = negativos_count.index.values
values = negativos_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Negativos')

py.offline.iplot([trace], filename='negativos')

In [64]:
ausentes_count = ausentes_por_bloque.agg('count')

labels = ausentes_count.index.values
values = ausentes_count.values.flatten()

trace = go.Pie(labels=labels, values=values, title='Votos Ausentes')

py.offline.iplot([trace], filename='ausentes')